Mnist数据集：http://yann.lecun.com/exdb/mnist

In [40]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [79]:
INPUT_NODE = 784 #输入层节点数，图片的像素

OUTPUT_NODE = 10 #输出的类别数目

LAYER1_NODE = 500 # 隐藏层节点数目

BATCH_SIZE = 100 #一个训练batch中的训练数据个数

LEARNING_RATE_BASE = 0.8 #基础的学习率

LEARNING_RATE_DECAY = 0.99 #学习率的衰减率

REGULARIZATION_RATE = 0.0001 #正则化项在损失函数中的系数

TRAINING_STEPS = 5000 # 训练论数

MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减率


In [80]:
#定义了使用relu的三层全连接神经网络的前向传播过程
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    if avg_class is None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [81]:
#训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, shape=[None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, shape=[None, OUTPUT_NODE], name="y-input")
    
    weight1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    weight2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
        
    #不使用滑动平均值
    y = inference(x, None, weight1, biases1, weight2, biases2)
    
    #存储训练轮数的变量，不需要使用滑动平均值，所以trainable=False
    global_step = tf.Variable(0, trainable=False)

    #初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    #对图上集合中没有指定trainable=False的参数应用滑动平均
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    #使用滑动平均值
    average_y = inference(x, variable_averages, weight1, biases1, weight2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weight1) + regularizer(weight2)
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, #基础学习率
                                               global_step, #当前迭代的轮数
                                               mnist.train.num_examples/BATCH_SIZE, # 过完所有训练数据需要的迭代次数
                                               LEARNING_RATE_DECAY,
                                              staircase=True)  #学习率衰减速度
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
     # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variables_average_op]):
        train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        #准备验证集
        validata_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
        #准备测试集
        test_feed = {x:mnist.test.images, y_:mnist.test.labels}
        
        #迭代训练神经网络
        for i in range(TRAINING_STEPS):
            if i% 1000 == 0:
                validata_acc = sess.run(accuracy, feed_dict=validata_feed)
                print("after %d training step, validata accuracy is %g" %(i, validata_acc))
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs, y_:ys})
        
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("after %d training step, test accuracy is %g" %(TRAINING_STEPS, test_acc))
        

In [82]:
def main(argv=None):
    mnist = input_data.read_data_sets("../data/MNIST_data", one_hot=True)
    train(mnist)

if __name__=='__main__':
    tf.app.run()

Extracting ../data/MNIST_data\train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data\t10k-labels-idx1-ubyte.gz
after 0 training step, validata accuracy is 0.1118
after 1000 training step, validata accuracy is 0.9784
after 2000 training step, validata accuracy is 0.9828
after 3000 training step, validata accuracy is 0.9836
after 4000 training step, validata accuracy is 0.9854
after 5000 training step, test accuracy is 0.9836


SystemExit: 

v = tf.Variable(初始化值， 名字)
v = tf.Variable(tf.constant(0.1, shape=[1]), name="v")

也可以使用tf.get_variable函数来创建或者获取变量
v = tf.get_variable("v", shape=[1], initializer=tf.constant_initializer(1.0))

当tf.variable_scope 参数reuse=True生成上下文管理器时，
tf.get_variable 获取已经创建的变量，如果变量不存在，报错；
当tf.variable_scope 参数reuse=False 或者为None，
tf.get_variable 如果变量已经存在，报错，否则创建新的变量；
reuse默认是false

在命名空间内创建的变量名称都会带上这个命名空间名作为前缀；

可以直接通过带命名空间名称的变量名来获取其他命名空间下的变量；
